In [ ]:
# TODO:
# Get data for Lower 48. See if another language will help us get data faster
# Put make_buffer, weather api calls and folium map in python module
# See if we can use mapclassify to color the pins


In [9]:
import folium
from folium.plugins import MarkerCluster
import numpy as np


# Bounding box around area of interest. Do entire lower 48 when you have time to run it. 
# Faster language might be ideal for this part? Timer?
west = -114.7
east = -102.3
north = 45.1
south = 33.1

# Resolution of grid of points
num_lat_points = 30
num_lon_points = 30

# Generate latitude and longitude points
latitudes = np.linspace(south, north, num_lat_points)
longitudes = np.linspace(west, east, num_lon_points)

m = folium.Map(location=[(north+south)/2,(east+west)/2], zoom_start=6)
cluster = MarkerCluster().add_to(m)

# Add generated coordinate pairs to cluster layer
for lat in latitudes:
    for lon in longitudes:
        folium.Marker([lat, lon]).add_to(cluster)

m

In [10]:
import geopandas as gpd
import requests
from shapely.geometry import Point

num_requests = 0

office_data = {
    'geometry' : [],
    'grid_x' : [],
    'grid_y' : [],
    'grid_id' : [],
    'original_point' : []
}
for lat in latitudes:
    for lng in longitudes:
        num_requests += 1
        print(f"{num_requests} requests made")
        response = requests.get(f'https://api.weather.gov/points/{lat},{lng}')
        if response.status_code != 200: print("unable to retrieve observation data")
        else:
            response_json = response.json()
            properties = response_json['properties']
            office_data['geometry'].append(Point(properties['relativeLocation']['geometry']['coordinates']))
            office_data['grid_x'].append(properties['gridX'])
            office_data['grid_y'].append(properties['gridY'])
            office_data['grid_id'].append(properties['gridId'])
            office_data['original_point'].append((lat,lng))
                                       
        
        

1 requests made
2 requests made
3 requests made
4 requests made
5 requests made
6 requests made
7 requests made
8 requests made
9 requests made
10 requests made
11 requests made
12 requests made
13 requests made
14 requests made
15 requests made
16 requests made
17 requests made
18 requests made
19 requests made
20 requests made
21 requests made
22 requests made
23 requests made
24 requests made
25 requests made
26 requests made
27 requests made
28 requests made
29 requests made
30 requests made
31 requests made
32 requests made
33 requests made
34 requests made
35 requests made
36 requests made
37 requests made
38 requests made
39 requests made
40 requests made
41 requests made
42 requests made
43 requests made
44 requests made
45 requests made
46 requests made
47 requests made
48 requests made
49 requests made
50 requests made
51 requests made
52 requests made
53 requests made
54 requests made
55 requests made
56 requests made
57 requests made
58 requests made
59 requests made
60 req

In [12]:
gdf_raw = gpd.GeoDataFrame(data=office_data)
gdf_dropna = gdf_raw.dropna(subset=['grid_id', 'grid_x', 'grid_y'])
gdf_clean = gdf_dropna.drop_duplicates()
len(gdf_clean)

891

In [27]:
from pyproj import Transformer, CRS
from shapely.geometry import Point
from shapely.ops import transform

def make_buffer(coordinates):

    aeqd_crs = CRS.from_proj4(f"+proj=aeqd +lat_0={lat} +lon_0={lon} +datum=WGS84")
    to_4326 = Transformer.from_crs(aeqd_crs, 4326, always_xy=True).transform
    to_aeqd = Transformer.from_crs(4326, aeqd_crs, always_xy=True).transform

    buffer_center = transform(to_aeqd, coordinates)
    buffer_aeqd = buffer_center.buffer(100000)  # 100 km
    buffer_4326 = transform(to_4326, buffer_aeqd)
    
    return gpd.GeoDataFrame(data={'geometry' : [buffer_4326]}, crs='EPSG:4326')

    

In [32]:
TEST_COORDS = Point(-107.9, 37.2)

buffer = make_buffer(TEST_COORDS)
forecastOffices = gdf_clean.set_crs('EPSG:4326')
points_within_buffer = gpd.sjoin(forecastOffices, buffer, how='inner', predicate='within')
points_within_buffer

,geometry,grid_x,grid_y,grid_id,original_point,index_right
252,POINT (-108.21864 36.65377),45,185,ABQ,"(36.41034482758621, -108.28620689655172)",0
281,POINT (-108.69941 36.79049),31,205,ABQ,"(36.824137931034485, -108.71379310344828)",0
282,POINT (-108.34571 36.74873),47,204,ABQ,"(36.824137931034485, -108.28620689655172)",0
283,POINT (-107.81419 36.73616),62,202,ABQ,"(36.824137931034485, -107.85862068965517)",0
284,POINT (-107.49308 36.99341),77,201,ABQ,"(36.824137931034485, -107.43103448275862)",0
285,POINT (-106.93537 36.93277),92,199,ABQ,"(36.824137931034485, -107.00344827586207)",0
311,POINT (-108.72646 37.21260),81,21,GJT,"(37.23793103448276, -108.71379310344828)",0
312,POINT (-108.29386 37.34660),96,19,GJT,"(37.23793103448276, -108.28620689655172)",0
313,POINT (-107.86892 37.27672),112,18,GJT,"(37.23793103448276, -107.85862068965517)",0
314,POINT (-107.59481 37.23525),127,16,GJT,"(37.23793103448276, -107.43103448275862)",0


In [45]:
import statistics


DESIRED_WEATHER = 70

points_within_buffer['mean_temp'] = np.nan
points_within_buffer['desired_temp_diff'] = np.nan

for idx, row in points_within_buffer.iterrows():
    print("Requesting", row['grid_x'], row['grid_y'], row['grid_id'])
    response_raw = requests.get(f"https://api.weather.gov/gridpoints/{row['grid_id']}/{row['grid_x']},{row['grid_y']}/forecast/hourly")
    if response_raw.status_code == 200:
        response = response_raw.json()
        periods = response['properties']['periods']
        ave_temps = statistics.mean([period['temperature'] for period in periods if 1 <= period['number'] <= 48])
        points_within_buffer.at[idx, 'mean_temp'] = ave_temps
        points_within_buffer.at[idx, 'desired_temp_diff'] = abs(ave_temps - DESIRED_WEATHER)

destinations_sorted = points_within_buffer.sort_values('desired_temp_diff')      
destinations_sorted 
    

Requesting 45 185 ABQ
Requesting 31 205 ABQ
Requesting 47 204 ABQ
Requesting 62 202 ABQ
Requesting 77 201 ABQ
Requesting 92 199 ABQ
Requesting 81 21 GJT
Requesting 96 19 GJT
Requesting 112 18 GJT
Requesting 127 16 GJT
Requesting 142 15 GJT
Requesting 22 25 PUB
Requesting 83 39 GJT
Requesting 98 38 GJT
Requesting 113 36 GJT
Requesting 129 35 GJT


,geometry,grid_x,grid_y,grid_id,original_point,index_right,mean_temp,desired_temp_diff
252,POINT (-108.21864 36.65377),45,185,ABQ,"(36.41034482758621, -108.28620689655172)",0,51.020833,18.979167
282,POINT (-108.34571 36.74873),47,204,ABQ,"(36.824137931034485, -108.28620689655172)",0,50.437500,19.562500
281,POINT (-108.69941 36.79049),31,205,ABQ,"(36.824137931034485, -108.71379310344828)",0,50.229167,19.770833
283,POINT (-107.81419 36.73616),62,202,ABQ,"(36.824137931034485, -107.85862068965517)",0,49.958333,20.041667
311,POINT (-108.72646 37.21260),81,21,GJT,"(37.23793103448276, -108.71379310344828)",0,49.104167,20.895833
284,POINT (-107.49308 36.99341),77,201,ABQ,"(36.824137931034485, -107.43103448275862)",0,49.000000,21.000000
312,POINT (-108.29386 37.34660),96,19,GJT,"(37.23793103448276, -108.28620689655172)",0,48.895833,21.104167
314,POINT (-107.59481 37.23525),127,16,GJT,"(37.23793103448276, -107.43103448275862)",0,47.875000,22.125000
341,POINT (-108.66006 37.50171),83,39,GJT,"(37.65172413793103, -108.71379310344828)",0,47.500000,22.500000
313,POINT (-107.86892 37.27672),112,18,GJT,"(37.23793103448276, -107.85862068965517)",0,46.708333,23.291667


In [89]:

# Create base map centered on your points
m2 = folium.Map(location=[points_within_buffer.geometry.y.mean(), points_within_buffer.geometry.x.mean()], zoom_start=6)


colors=['blue', 'red']

# Loop through each point
for idx, row in points_within_buffer.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Mean Temp: {row['mean_temp']:.1f}°F",
        icon=folium.Icon(color=colors[0] if row['mean_temp'] < 47 else colors[1], icon='cloud')
    ).add_to(m2)

m2